# Machine learning - Assignment 4 - Classification
____
**Author**: Kemal Cikota

**Course**: Machine learning
____

## Introduction

In this assignment, we were first tasked with answeriing and discussing differences between classification models, such as, Linear Discriminant Analysis (LDA), Quadratic Discriminant Analysis (QDA), and k-nearest neighbor (KNN). The later practical part of this assignment involves implementing using the 'Smarket.csv' dataset which is a financial dataset that containst stock market returns. The classification models will be used to predict market movements. Conclusions and interpretations of the outputs will also be made in this notebook to make it clearer as to how and why certain calculations are computed and what their results indicate.

For many of the explanations i will use [this source](https://rdrr.io/cran/ISLR/man/Smarket.html) as a reference for what the columns means. Theese short descriptions were of great help to me.

## Conceptual Questions

**1. Discuss the differences between LDA and QDA in terms of their main assumptions
about classes, decision boundaries, number of samples, and overfitting**

The main assumption that LDA makes about classes is that LDA assumes that the different classes follow a _multivariate normal distribution with a common covaraince matrix_. In this case, a class is a 'value' that a classification variable/column/feature can take. So for example, the two classes in the smarket.csv file can be found under the **Direction** column and they are **up** and **down**. But what do i mean by common covariance matrix? Well, a covariance matrix is a square matrix that contains the covariances between all pairs of features. It is used to model the distribution of the data and to determine the shape of decision variables in classification. On the other hand, QDA makes the assumption that each class should have its own covariance matrix so for the smarket dataset, this would mean that the classes **up** and **down** have their own covariance matrix. In practice, it would mean that the covariance matrix for LDA would look something like this (not a guarantee that this is 100% correct, its more of a 'proof of concept').

$$
\Sigma =
\begin{bmatrix}
\text{Var}(\text{Lag1}) & \text{Cov}(\text{Lag1}, \text{Lag2}) & \text{Cov}(\text{Lag1}, \text{Lag3}) & \text{Cov}(\text{Lag1}, \text{Volume}) \\
\text{Cov}(\text{Lag2}, \text{Lag1}) & \text{Var}(\text{Lag2}) & \text{Cov}(\text{Lag2}, \text{Lag3}) & \text{Cov}(\text{Lag2}, \text{Volume}) \\
\text{Cov}(\text{Lag3}, \text{Lag1}) & \text{Cov}(\text{Lag3}, \text{Lag2}) & \text{Var}(\text{Lag3}) & \text{Cov}(\text{Lag3}, \text{Volume}) \\
\text{Cov}(\text{Volume}, \text{Lag1}) & \text{Cov}(\text{Volume}, \text{Lag2}) & \text{Cov}(\text{Volume}, \text{Lag3}) & \text{Var}(\text{Volume})
\end{bmatrix}
$$

But for QDA we have _two_ matrices, one for each class.

$$
\Sigma{\text{Up}} =
\begin{bmatrix}
\text{Var}(\text{Lag1}) & \text{Cov}(\text{Lag1}, \text{Lag2}) & \cdots \\
\text{Cov}(\text{Lag2}, \text{Lag1}) & \text{Var}(\text{Lag2}) & \cdots \\
\vdots & \vdots & \ddots
\end{bmatrix}
$$

$$
\Sigma{\text{Down}} =
\begin{bmatrix}
\text{Var}(\text{Lag1}) & \text{Cov}(\text{Lag1}, \text{Lag2}) & \cdots \\
\text{Cov}(\text{Lag2}, \text{Lag1}) & \text{Var}(\text{Lag2}) & \cdots \\
\vdots & \vdots & \ddots
\end{bmatrix}
$$

Now when this is clear, i can now discuss the assumptions about decision boundaries, number of samples and overfitting as all of them are kind of an effect from the fact that the covaraince matrices are different like this.

The decision boundaries for LDA is always a linear hyperplane. So it can either be a straight line if it's in 2D or it can be a straight plane if it's in 3D (i mean it can "tilt" but the plane itself can not bend or curve). This makes sence since all classes share the same covariance structure. However, in QDA, we allow each class to have its own covariance matrix, so the decision bounds is quadratic, meaning it can be a curved (non-linear) hyperplane in 3D. This means that LDA is best suited for problems where the class distributions are linearly separable.

LDA requires fewer training samples since it estimates only one covariance matrix shared across all classes while QDA requires significantly more training data because it must estimate a separate covariance matrix for each class. We can also see that this relationship can be mathematically shown since LDA requires $d(d + 1)/2$ for d predictor variables while QDA needs $C * (d(d + 1)/2)$ parameters where $C$ is the amount of classes. So, as we can see, this would scale very aggresively if we have a lot of classes in our data since we would have to multiply the amount of classes in QDA as opposed to LDA where we only take the predictors in to account.

When it comes to overfitting, i would say that LDA probably has a less chance to overfit on some generic dataset because strong assumptions are made about the distribution of classes (same covariance matrices/structure) and uses much less parameters especially for data with a lot of classes. But QDA is probably more prone to overfitting, especially with less data, because it models each class separately with more parameters.

**2. Regarding KNN**

**a) How does the choice of distance metric affect the performance of KNN classification?**

In the KNN (K- nearest neighbor) algorithm, the algorithms classifies a new data point by looking at the 'k' closest training points and assigning the majorit class among them. The way the distance metric is calculated (basically, the range of what to include in K) will directly impact what is considered a 'neighbor' which also impacts how accurate the classification is and how sensitive the KNN algorithm is to certain features.

There are actually a lot of different ways of calculating the distance metric since it is not that different from calculating any other distance in mathematics. The way of calculating distance that i have learnt from mainly linear algebra but also numerical methods and optimization courses and that i am most familiar with is _euclidean distance_ which is just the line segment between two points. It is easily and intuitively calculated like this: $d(x, y) = \sqrt{\sum(x_i - y_i)^2}$

This formula measures straight line distances works well when all features have the same scale and are independent of each other which makes it applicable for a wide range of simpler data but if we have data features with different scales (like age in years and salary in thousands of dollars), the salary will dominate the age by a lot so we would have to find a way to normalize our data to [0, 1] ranges in order to prevent features to overwhelm others.

There are also other ways of calculating distances but many of them have their pro's and cons so chosing a method really depends on the data itself.

**b) please also discuss the concept of the curse of dimensionality and its implications for KNN algorithm**

The curse of dimensionality is a problem when working with data that has a lot of dimensions (features) and data points become more and more sparse. This makes it difficult for distance based learning algorithms (like KNN) to work efficiently. This is because in high dimensional spaces, the distance between points tend to be further away from each other which makes it so that the meaning of _closeness_ becomes meaningless and the distance metrics become less and less reliable.


## Practical

Short intro about what is about to happen.

### Load the data and get an overview of the data

In [ ]:
import pandas as pd # Never coded in R before but this seems to be the equivalent of library(pandas) in R
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

# load Smarket.csv
boston = pd.read_csv('Smarket.csv')

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)